In [1]:
import logging
import threading
from multiprocessing import Process
from time import sleep, time
import numpy as np
from kazoo.client import KazooClient
from kazoo.protocol.paths import join

In [2]:
zk = KazooClient()
zk.start()
zk.stop()

In [3]:
class Participant(Process):
    
    def __init__(self, root: str, _id: int):
        
        super().__init__()
        self.url = f'{root}/{_id}'
        self.root = root
        self.id = _id

    
        zk = KazooClient()
        zk.start()
        
        val = np.random.random()
        if val > 0.5:
            value = b'commit' 
        else:
            value = b'abort'
        
        print(f'Participant {self.id} request {value.decode()}')
        
        zk.create(self.url, value, ephemeral=True)
        
        @zk.DataWatch(self.url)
        def watch_myself(data, stat):
            if stat.version != 0:
                print(f'Participant {self.id} do {data.decode()}')

        sleep(5)
        zk.stop()
        zk.close()

In [4]:
class Transaction_Coordinator():
    def main(self):
        coordinator = KazooClient()
        coordinator.start()

        if coordinator.exists('/data'):
            coordinator.delete('/data', recursive=True)

        coordinator.create('/data')
        coordinator.create('/data/tc')

        num = 5
        N = 30
        self.timer = None

        def check_participants():
            participants = coordinator.get_children('/data/tc')
            commit_count = 0
            abort_count = 0
                       
            for participant in participants:
                commit_counter += int(coordinator.get(f'/data/tc/{participant}')[0] == b'commit')
                abort_counter += int(coordinator.get(f'/data/tc/{participant}')[0] == b'abort')

            if commit_count > abort_count:
                target = b'commit' 
            else:
                target = b'abort'
            
            for participant in participants:
                coordinator.set(f'/data/tc/{participant}', target)

        @coordinator.ChildrenWatch('/data/tc')
        def watch_participants(participants):
            if len(participants) == 0:
                if self.timer is not None:
                    self.timer.cancel()
                    
            else:
                if self.timer is not None:
                    self.timer.cancel()

                timer = threading.Timer(N, check_participants)
                timer.daemon = True
                timer.start()
                

            if len(participants) < num:
                print('waiting...')
                            
            elif len(participants) == num:
                print('check')
                timer.cancel()
                check_participants()
                print(len(participants))
                

        root = '/data/tc'
        for i in range(5):
            p = Participant(root, i)
            p.start()

In [5]:
Transaction_Coordinator().main()

waiting...
Participant 0 request commit
waiting...
waiting...
Participant 1 request abort
waiting...
waiting...
Participant 2 request commit
waiting...
waiting...
Participant 3 request commit
waiting...
waiting...
Participant 4 request commit
waiting...
waiting...
